<h2>Introducing miscellaneous utilities in AWOT.</h2>
<h4>This notebook will grow over time as utilites are added and I have time to update.</h4>

In [2]:
# Load the needed packages
import os
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset

import awot

%matplotlib inline

<b> First we'll need some data to interact with</b>

In [2]:
# Released data file
wcrf1 = os.path.join("/Users/guy/data/king_air/owles2013/wcr", "WCR.OWLES13.20131215.225944_234806.up-down.nc")

# Supplementary file with corrected velocity data
wcrf2 = os.path.join("/Users/guy/data/king_air/owles2013/wcr/", "W-CORRECTED.WCR.OWLES13.20131215.225944_234806.up-down.nc")

<b>Read in the radar data</b>

In [3]:
wcr = awot.io.read_wcr2(fname=wcrf1)

height_zdr does not exist in file...
height_ldr does not exist in file...
zdr does not exist in file...
ldr does not exist in file...


<h3>Data instances</h3>

<b>Read a variable from another file and add it to the AWOT dictionary. A mask of invalid data is automatically applied. Additional masking can be accomplished by setting the <i>mask_value</i> keyword.</b>

In [4]:
nc = Dataset(wcrf2)
velcor = nc.variables['Velocity_cor_2']
awot.util.add_dict_to_awot_fields(wcr, 'velocity_corrected', data=velcor[:],
                                 units=velcor.units, longname=velcor.long_name, stdname="Corrected velocity")

print(wcr['fields']['velocity']['data'].shape, wcr['fields']['velocity_corrected']['data'].shape)
print(np.ma.min(wcr['fields']['velocity']['data']), np.ma.max(wcr['fields']['velocity']['data']))
print(np.ma.min(wcr['fields']['velocity_corrected']['data']), np.ma.max(wcr['fields']['velocity_corrected']['data']))

((14388, 379), (14388, 379))
(-15.640082, 15.023235)
(-16.568499, 19.136526)


<b>Just as in the plotting routines, time can be subset using a date string. But maybe you want to just return a subsetted dicationary for use. The <i>time_subset_awot_dict</i> function does this.</b>

In [7]:
start_time = "2013-12-15 23:05:00"
end_time = "2013-12-15 23:15:00"

# Create subsets of arrays
refsub = awot.util.time_subset_awot_dict(wcr['time'], wcr['fields']['reflectivity'],
                                        start_time, end_time)
velsub = awot.util.time_subset_awot_dict(wcr['time'], wcr['fields']['velocity'],
                                        start_time, end_time)
altsub = awot.util.time_subset_awot_dict(wcr['time'], wcr['altitude'],
                                        start_time, end_time)

In [8]:
print(wcr['fields']['reflectivity']['data'].shape, refsub['data'].shape)
print(wcr['fields']['velocity']['data'].shape, velsub['data'].shape)
print(wcr['altitude']['data'].shape, altsub['data'].shape)

((14388, 379), (2975, 379))
((14388, 379), (2975, 379))
((14388,), (2975,))


<b>AWOT uses Py-ART to read many radar files. Therefore we can read through AWOT.</b>

In [8]:
nexf = os.path.join("/Users/guy/data/nexrad/KILN/nex2/20140429", "KILN20140430_004708_V06")
rnex = awot.io.read_ground_radar(nexf, map_to_awot=False)

In [7]:
rnex.fields.keys()

/Users/guy/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:90: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/Users/guy/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:667: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):


['differential_phase',
 'cross_correlation_ratio',
 'spectrum_width',
 'reflectivity',
 'differential_reflectivity',
 'velocity']

<b> By changing the <i>map_to_awot</i> key we can convert the Py-ART radar instance to an AWOT radar instance. Note this is the DEFAULT behavior to make working with the AWOT package a bit easier.</b>

In [9]:
rnex = awot.io.read_ground_radar(nexf, map_to_awot=True)

In [10]:
rnex.keys()

['platform',
 'time',
 'instrument',
 'latitude',
 'fields',
 'height',
 'data_format',
 'longitude',
 'metadata']

<b>An experimental KMZ file creation is available for flight data.</b>

In [11]:
flname = os.path.join("/Users/guy/data/king_air/pecan2015", "20150716.c1.nc")
fl1 = awot.io.read_netcdf(fname=flname, platform='uwka')

Using 'time' to make AWOT time variable


<b>Now we can create a KMZ file of the track. This saves a KMZ file to current working directory if not specified.</b>

In [ ]:
awot.util.write_track_kmz(fl1, 'altitude', show_legend=False, end_time="2016-01-01T00:00:00")